In [3]:
import pandas as pd
import numpy as np
import datetime
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [53]:
def Recall(df_true_pred, threshold, flag):
    """
        df_true_pred : 'user_id', 'cate', 'shop_id', 'label', 'pred_prob'
            flag : 'user_cate' or 'user_cate_shop'
            Threshold = 0.5
            Recall = TP / (TP + FN)
    """
    temp_ = df_true_pred[df_true_pred['pred_prob']>=threshold]
    if flag == 'user_cate':
        temp_ = temp_.drop_duplicates(['user_id', 'cate'])
        recall = np.sum(temp_['label']) * 1.0 / np.sum(df_true_pred['label'])
    elif flag == 'user_cate_shop':
        recall = np.sum(temp_['label']) * 1.0 / np.sum(df_true_pred['label'])
    else:
        recall = -1
    return recall
	
def Precision(df_true_pred, threshold, flag):
    """
        df_true_pred : 'user_id', 'cate', 'shop_id', 'label', 'pred_prob'
            flag : 'user_cate' or 'user_cate_shop'
            Threshold 
            Precision = TP / (TP + FP)
    """
    temp_ = df_true_pred[df_true_pred['pred_prob']>=threshold]
    if flag == 'user_cate':
        temp_ = temp_.drop_duplicates(['user_id', 'cate'])
        precision = np.sum(temp_['label']) * 1.0 / np.size(temp_)
    elif flag == 'user_cate_shop':
        precision = np.sum(temp_['label']) * 1.0 / np.size(temp_)
    else:
        precision = -1
    return precision

def get_metrice(df_true_pred, threshold):
    """
        df_true_pred : 'user_id', 'cate', 'shop_id', 'label', 'pred_prob'
        Threshold = 0.5
    """ 

    # 用户-品类
    R1_1 = Recall(df_true_pred, threshold, flag='user_cate') 
    P1_1 = Precision(df_true_pred, threshold, flag='user_cate')
    F1_1 = 3 * R1_1 * P1_1 / (2 * R1_1 + P1_1)

    # 用户-品类-店铺
    R1_2 = Recall(df_true_pred, threshold, flag='user_cate_shop') 
    P1_2 = Precision(df_true_pred, threshold, flag='user_cate_shop')
    F1_2 = 5 * R1_2 * P1_2 / (2 * R1_2 + 3 * P1_2)

    # 总分
    score = 0.4 * F1_1 + 0.6 * F1_2
    return score

In [ ]:


# 模型工具
class SBBTree():
    """Stacking,Bootstap,Bagging----SBBTree"""
    """ author：Cookly 洪鹏飞 """
    def __init__(self, params, stacking_num, bagging_num, bagging_test_size, num_boost_round, early_stopping_rounds):
        """
        Initializes the SBBTree.
        Args:
          params : lgb params.
          stacking_num : k_flod stacking.
          bagging_num : bootstrap num.
          bagging_test_size : bootstrap sample rate.
          num_boost_round : boost num.
          early_stopping_rounds : early_stopping_rounds.
        """
        self.params = params
        self.stacking_num = stacking_num
        self.bagging_num = bagging_num
        self.bagging_test_size = bagging_test_size
        self.num_boost_round = num_boost_round
        self.early_stopping_rounds = early_stopping_rounds

        self.model = lgb
        self.stacking_model = []
        self.bagging_model = []

    def fit(self, X, y):
        """ fit model. """
        if self.stacking_num > 1:
            layer_train = np.zeros((X.shape[0], 2))
            self.SK = StratifiedKFold(n_splits=self.stacking_num, shuffle=True, random_state=1)
            for k,(train_index, test_index) in enumerate(self.SK.split(X, y)):
                X_train = X[train_index]
                y_train = y[train_index]
                X_test = X[test_index]
                y_test = y[test_index]

                lgb_train = lgb.Dataset(X_train, y_train)
                lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

                gbm = lgb.train(self.params,
                            lgb_train,
                            num_boost_round=self.num_boost_round,
                            valid_sets=lgb_eval,
                            early_stopping_rounds=self.early_stopping_rounds)

                self.stacking_model.append(gbm)

                pred_y = gbm.predict(X_test, num_iteration=gbm.best_iteration)
                layer_train[test_index, 1] = pred_y

            X = np.hstack((X, layer_train[:,1].reshape((-1,1)))) 
        else:
            pass
        for bn in range(self.bagging_num):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.bagging_test_size, random_state=bn)

            lgb_train = lgb.Dataset(X_train, y_train)
            lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

            gbm = lgb.train(self.params,
                        lgb_train,
                        num_boost_round=2000,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=200)

            self.bagging_model.append(gbm)

    def predict(self, X_pred):
        """ predict test data. """
        if self.stacking_num > 1:
            test_pred = np.zeros((X_pred.shape[0], self.stacking_num))
            for sn,gbm in enumerate(self.stacking_model):
                pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
                test_pred[:, sn] = pred
            X_pred = np.hstack((X_pred, test_pred.mean(axis=1).reshape((-1,1))))  
        else:
            pass 
        for bn,gbm in enumerate(self.bagging_model):
            pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
            if bn == 0:
                pred_out=pred
            else:
                pred_out+=pred
        return pred_out/self.bagging_num

	# 模型参数
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'num_leaves': 2 ** 5 - 1,
    'min_child_samples': 100,
    'max_bin': 100,
    'subsample': .7,
    'subsample_freq': 1,
    'colsample_bytree': 0.7,
    'min_child_weight': 0,
    'scale_pos_weight': 30,
    'seed': 2018,
    'nthread': 30,
    'verbose': 0,
}
#读取特征名称
train_test_columns=pd.read_csv('train_test_columns.csv')
train_test_columns=np.array(train_test_columns['features'])
#读取训练集
train_set=pd.read_csv('train_set.csv')	
train_metrics = train_set[['user_id','cate','shop_id','label']]
X_train = train_set[train_test_columns].values
y_train = train_set['label'].values
del train_set #释放变量

#读取测试集
test_set=pd.read_csv('test_set.csv')
X_test = test_set[train_test_columns].values
submit_ID = test_set[['user_id','cate','shop_id']]
del test_set







[1]	valid_0's auc: 0.860374
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.871563
[3]	valid_0's auc: 0.875348
[4]	valid_0's auc: 0.877528
[5]	valid_0's auc: 0.879847
[6]	valid_0's auc: 0.88071
[7]	valid_0's auc: 0.881397
[8]	valid_0's auc: 0.882404
[9]	valid_0's auc: 0.883001
[10]	valid_0's auc: 0.883304
Did not meet early stopping. Best iteration is:
[10]	valid_0's auc: 0.883304
[1]	valid_0's auc: 0.864607
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.876486
[3]	valid_0's auc: 0.879786
[4]	valid_0's auc: 0.881721
[5]	valid_0's auc: 0.883395
[6]	valid_0's auc: 0.884464
[7]	valid_0's auc: 0.885482
[8]	valid_0's auc: 0.886293
[9]	valid_0's auc: 0.887066
[10]	valid_0's auc: 0.887495
Did not meet early stopping. Best iteration is:
[10]	valid_0's auc: 0.887495
[1]	valid_0's auc: 0.863689
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.875082
[3]	valid_0's auc: 0.880155
[4]	valid_0'

In [68]:
# 使用模型
model = SBBTree(params=params,\
                      stacking_num=5,\
                      bagging_num=5,\
                      bagging_test_size=0.33,\
                      num_boost_round=100,\
                      early_stopping_rounds=20)
model.fit(X_train, y_train)
#y_predict = model.predict(X_test)
#y_train_predict = model.predict(X_train)

[1]	valid_0's auc: 0.860374
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.871563
[3]	valid_0's auc: 0.875348
[4]	valid_0's auc: 0.877528
[5]	valid_0's auc: 0.879847
[6]	valid_0's auc: 0.88071
[7]	valid_0's auc: 0.881397
[8]	valid_0's auc: 0.882404
[9]	valid_0's auc: 0.883001
[10]	valid_0's auc: 0.883304
[11]	valid_0's auc: 0.883992
[12]	valid_0's auc: 0.884255
[13]	valid_0's auc: 0.884575
[14]	valid_0's auc: 0.885007
[15]	valid_0's auc: 0.88523
[16]	valid_0's auc: 0.885644
[17]	valid_0's auc: 0.885891
[18]	valid_0's auc: 0.886151
[19]	valid_0's auc: 0.886402
[20]	valid_0's auc: 0.886713
[21]	valid_0's auc: 0.886834
[22]	valid_0's auc: 0.887001
[23]	valid_0's auc: 0.887144
[24]	valid_0's auc: 0.887222
[25]	valid_0's auc: 0.887339
[26]	valid_0's auc: 0.887409
[27]	valid_0's auc: 0.887596
[28]	valid_0's auc: 0.887721
[29]	valid_0's auc: 0.88785
[30]	valid_0's auc: 0.887969
[31]	valid_0's auc: 0.888115
[32]	valid_0's auc: 0.888228
[33]	valid_0's auc: 0.

[74]	valid_0's auc: 0.895182
[75]	valid_0's auc: 0.895286
[76]	valid_0's auc: 0.895328
[77]	valid_0's auc: 0.895368
[78]	valid_0's auc: 0.895423
[79]	valid_0's auc: 0.895476
[80]	valid_0's auc: 0.895532
[81]	valid_0's auc: 0.895605
[82]	valid_0's auc: 0.895644
[83]	valid_0's auc: 0.895705
[84]	valid_0's auc: 0.895754
[85]	valid_0's auc: 0.895801
[86]	valid_0's auc: 0.89584
[87]	valid_0's auc: 0.895875
[88]	valid_0's auc: 0.895897
[89]	valid_0's auc: 0.895935
[90]	valid_0's auc: 0.895947
[91]	valid_0's auc: 0.895984
[92]	valid_0's auc: 0.89605
[93]	valid_0's auc: 0.896089
[94]	valid_0's auc: 0.896127
[95]	valid_0's auc: 0.896155
[96]	valid_0's auc: 0.896191
[97]	valid_0's auc: 0.896218
[98]	valid_0's auc: 0.896242
[99]	valid_0's auc: 0.896264
[100]	valid_0's auc: 0.896306
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.896306
[1]	valid_0's auc: 0.863852
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.872912
[3]	valid_0's auc: 0.8

[44]	valid_0's auc: 0.895109
[45]	valid_0's auc: 0.895153
[46]	valid_0's auc: 0.895151
[47]	valid_0's auc: 0.895182
[48]	valid_0's auc: 0.895211
[49]	valid_0's auc: 0.89525
[50]	valid_0's auc: 0.895314
[51]	valid_0's auc: 0.895319
[52]	valid_0's auc: 0.895326
[53]	valid_0's auc: 0.895361
[54]	valid_0's auc: 0.895381
[55]	valid_0's auc: 0.895397
[56]	valid_0's auc: 0.895421
[57]	valid_0's auc: 0.895441
[58]	valid_0's auc: 0.89546
[59]	valid_0's auc: 0.895493
[60]	valid_0's auc: 0.895506
[61]	valid_0's auc: 0.895555
[62]	valid_0's auc: 0.895569
[63]	valid_0's auc: 0.895621
[64]	valid_0's auc: 0.895645
[65]	valid_0's auc: 0.895657
[66]	valid_0's auc: 0.89568
[67]	valid_0's auc: 0.895723
[68]	valid_0's auc: 0.895741
[69]	valid_0's auc: 0.895777
[70]	valid_0's auc: 0.895795
[71]	valid_0's auc: 0.895809
[72]	valid_0's auc: 0.895833
[73]	valid_0's auc: 0.895851
[74]	valid_0's auc: 0.895853
[75]	valid_0's auc: 0.895865
[76]	valid_0's auc: 0.895887
[77]	valid_0's auc: 0.895892
[78]	valid_0's au

[320]	valid_0's auc: 0.898439
[321]	valid_0's auc: 0.898447
[322]	valid_0's auc: 0.898458
[323]	valid_0's auc: 0.898461
[324]	valid_0's auc: 0.898471
[325]	valid_0's auc: 0.898483
[326]	valid_0's auc: 0.898489
[327]	valid_0's auc: 0.898492
[328]	valid_0's auc: 0.898498
[329]	valid_0's auc: 0.898505
[330]	valid_0's auc: 0.898519
[331]	valid_0's auc: 0.898516
[332]	valid_0's auc: 0.898522
[333]	valid_0's auc: 0.898528
[334]	valid_0's auc: 0.89854
[335]	valid_0's auc: 0.898539
[336]	valid_0's auc: 0.89854
[337]	valid_0's auc: 0.898545
[338]	valid_0's auc: 0.898549
[339]	valid_0's auc: 0.898548
[340]	valid_0's auc: 0.898549
[341]	valid_0's auc: 0.898551
[342]	valid_0's auc: 0.898546
[343]	valid_0's auc: 0.89855
[344]	valid_0's auc: 0.898553
[345]	valid_0's auc: 0.89856
[346]	valid_0's auc: 0.898568
[347]	valid_0's auc: 0.898577
[348]	valid_0's auc: 0.898575
[349]	valid_0's auc: 0.898575
[350]	valid_0's auc: 0.898593
[351]	valid_0's auc: 0.898606
[352]	valid_0's auc: 0.898619
[353]	valid_0'

[595]	valid_0's auc: 0.89956
[596]	valid_0's auc: 0.899566
[597]	valid_0's auc: 0.899563
[598]	valid_0's auc: 0.899566
[599]	valid_0's auc: 0.899572
[600]	valid_0's auc: 0.899567
[601]	valid_0's auc: 0.899577
[602]	valid_0's auc: 0.899581
[603]	valid_0's auc: 0.899588
[604]	valid_0's auc: 0.899593
[605]	valid_0's auc: 0.89959
[606]	valid_0's auc: 0.899597
[607]	valid_0's auc: 0.899602
[608]	valid_0's auc: 0.899607
[609]	valid_0's auc: 0.899607
[610]	valid_0's auc: 0.899612
[611]	valid_0's auc: 0.899621
[612]	valid_0's auc: 0.899629
[613]	valid_0's auc: 0.899635
[614]	valid_0's auc: 0.899633
[615]	valid_0's auc: 0.899633
[616]	valid_0's auc: 0.899636
[617]	valid_0's auc: 0.899638
[618]	valid_0's auc: 0.899641
[619]	valid_0's auc: 0.899644
[620]	valid_0's auc: 0.899648
[621]	valid_0's auc: 0.899654
[622]	valid_0's auc: 0.899657
[623]	valid_0's auc: 0.899664
[624]	valid_0's auc: 0.899667
[625]	valid_0's auc: 0.899677
[626]	valid_0's auc: 0.899677
[627]	valid_0's auc: 0.899675
[628]	valid_

...

[694]	valid_0's auc: 0.898622
[695]	valid_0's auc: 0.898627
[696]	valid_0's auc: 0.898637
[697]	valid_0's auc: 0.898639
[698]	valid_0's auc: 0.898646
[699]	valid_0's auc: 0.89865
[700]	valid_0's auc: 0.898653
[701]	valid_0's auc: 0.898662
[702]	valid_0's auc: 0.898664
[703]	valid_0's auc: 0.898668
[704]	valid_0's auc: 0.898667
[705]	valid_0's auc: 0.898673
[706]	valid_0's auc: 0.898676
[707]	valid_0's auc: 0.898674
[708]	valid_0's auc: 0.898673
[709]	valid_0's auc: 0.898672
[710]	valid_0's auc: 0.898682
[711]	valid_0's auc: 0.898686
[712]	valid_0's auc: 0.89869
[713]	valid_0's auc: 0.898694
[714]	valid_0's auc: 0.89869
[715]	valid_0's auc: 0.898687
[716]	valid_0's auc: 0.898699
[717]	valid_0's auc: 0.898706
[718]	valid_0's auc: 0.89871
[719]	valid_0's auc: 0.898713
[720]	valid_0's auc: 0.898714
[721]	valid_0's auc: 0.898711
[722]	valid_0's auc: 0.898716
[723]	valid_0's auc: 0.898714
[724]	valid_0's auc: 0.898714
[725]	valid_0's auc: 0.898715
[726]	valid_0's auc: 0.898718
[727]	valid_0'

[969]	valid_0's auc: 0.899363
[970]	valid_0's auc: 0.899356
[971]	valid_0's auc: 0.899363
[972]	valid_0's auc: 0.899356
[973]	valid_0's auc: 0.899352
[974]	valid_0's auc: 0.899356
[975]	valid_0's auc: 0.899359
[976]	valid_0's auc: 0.899354
[977]	valid_0's auc: 0.899355
[978]	valid_0's auc: 0.899357
[979]	valid_0's auc: 0.899359
[980]	valid_0's auc: 0.899354
[981]	valid_0's auc: 0.899347
[982]	valid_0's auc: 0.899351
[983]	valid_0's auc: 0.899354
[984]	valid_0's auc: 0.899356
[985]	valid_0's auc: 0.899357
[986]	valid_0's auc: 0.899359
[987]	valid_0's auc: 0.89936
[988]	valid_0's auc: 0.899363
[989]	valid_0's auc: 0.899371
[990]	valid_0's auc: 0.899369
[991]	valid_0's auc: 0.899372
[992]	valid_0's auc: 0.899369
[993]	valid_0's auc: 0.899371
[994]	valid_0's auc: 0.899373
[995]	valid_0's auc: 0.899373
[996]	valid_0's auc: 0.899377
[997]	valid_0's auc: 0.899378
[998]	valid_0's auc: 0.899376
[999]	valid_0's auc: 0.899374
[1000]	valid_0's auc: 0.899377
[1001]	valid_0's auc: 0.899381
[1002]	va

[1236]	valid_0's auc: 0.899711
[1237]	valid_0's auc: 0.899712
[1238]	valid_0's auc: 0.899714
[1239]	valid_0's auc: 0.899715
[1240]	valid_0's auc: 0.899722
[1241]	valid_0's auc: 0.89972
[1242]	valid_0's auc: 0.899724
[1243]	valid_0's auc: 0.899724
[1244]	valid_0's auc: 0.899728
[1245]	valid_0's auc: 0.899726
[1246]	valid_0's auc: 0.899732
[1247]	valid_0's auc: 0.899736
[1248]	valid_0's auc: 0.899736
[1249]	valid_0's auc: 0.899743
[1250]	valid_0's auc: 0.899748
[1251]	valid_0's auc: 0.899749
[1252]	valid_0's auc: 0.89975
[1253]	valid_0's auc: 0.899751
[1254]	valid_0's auc: 0.899763
[1255]	valid_0's auc: 0.899764
[1256]	valid_0's auc: 0.899769
[1257]	valid_0's auc: 0.899773
[1258]	valid_0's auc: 0.899777
[1259]	valid_0's auc: 0.899774
[1260]	valid_0's auc: 0.899776
[1261]	valid_0's auc: 0.89978
[1262]	valid_0's auc: 0.899786
[1263]	valid_0's auc: 0.899786
[1264]	valid_0's auc: 0.89978
[1265]	valid_0's auc: 0.899787
[1266]	valid_0's auc: 0.89979
[1267]	valid_0's auc: 0.899788
[1268]	valid_

[1502]	valid_0's auc: 0.900044
[1503]	valid_0's auc: 0.900046
[1504]	valid_0's auc: 0.90004
[1505]	valid_0's auc: 0.900039
[1506]	valid_0's auc: 0.900038
[1507]	valid_0's auc: 0.90004
[1508]	valid_0's auc: 0.900037
[1509]	valid_0's auc: 0.900039
[1510]	valid_0's auc: 0.900035
[1511]	valid_0's auc: 0.900044
[1512]	valid_0's auc: 0.900045
[1513]	valid_0's auc: 0.900042
[1514]	valid_0's auc: 0.900046
[1515]	valid_0's auc: 0.900047
[1516]	valid_0's auc: 0.900054
[1517]	valid_0's auc: 0.900055
[1518]	valid_0's auc: 0.900054
[1519]	valid_0's auc: 0.900056
[1520]	valid_0's auc: 0.900053
[1521]	valid_0's auc: 0.900055
[1522]	valid_0's auc: 0.900057
[1523]	valid_0's auc: 0.900066
[1524]	valid_0's auc: 0.900068
[1525]	valid_0's auc: 0.900073
[1526]	valid_0's auc: 0.900072
[1527]	valid_0's auc: 0.900074
[1528]	valid_0's auc: 0.900074
[1529]	valid_0's auc: 0.900075
[1530]	valid_0's auc: 0.900076
[1531]	valid_0's auc: 0.90007
[1532]	valid_0's auc: 0.900066
[1533]	valid_0's auc: 0.90007
[1534]	valid

[1768]	valid_0's auc: 0.900221
[1769]	valid_0's auc: 0.900226
[1770]	valid_0's auc: 0.900224
[1771]	valid_0's auc: 0.900223
[1772]	valid_0's auc: 0.900222
[1773]	valid_0's auc: 0.900223
[1774]	valid_0's auc: 0.900221
[1775]	valid_0's auc: 0.900224
[1776]	valid_0's auc: 0.900218
[1777]	valid_0's auc: 0.90022
[1778]	valid_0's auc: 0.900223
[1779]	valid_0's auc: 0.90022
[1780]	valid_0's auc: 0.900222
[1781]	valid_0's auc: 0.900218
[1782]	valid_0's auc: 0.900216
[1783]	valid_0's auc: 0.900214
[1784]	valid_0's auc: 0.900216
[1785]	valid_0's auc: 0.900214
[1786]	valid_0's auc: 0.900215
[1787]	valid_0's auc: 0.900212
[1788]	valid_0's auc: 0.900214
[1789]	valid_0's auc: 0.900218
[1790]	valid_0's auc: 0.90022
[1791]	valid_0's auc: 0.900221
[1792]	valid_0's auc: 0.900226
[1793]	valid_0's auc: 0.900229
[1794]	valid_0's auc: 0.90023
[1795]	valid_0's auc: 0.900233
[1796]	valid_0's auc: 0.900233
[1797]	valid_0's auc: 0.900235
[1798]	valid_0's auc: 0.900243
[1799]	valid_0's auc: 0.900249
[1800]	valid

In [84]:
# 线下验证公式 ('user_id', 'cate', 'shop_id', 'label', 'pred_prob')
train_metrics['pred_prob'] = y_train_predict
scores = get_metrice(train_metrics, threshold=0.6)
print("线下得分：",scores)


线下得分： 0.07026545357609426


In [87]:
#### 结果输出
submit_ID['pred_prob'] = y_predict
submit_ID[submit_ID['pred_prob']>=0.76][['user_id','cate','shop_id']].to_csv('submit_baseline4.csv',header=True,sep=',',encoding='utf-8',index=False)

In [49]:
train_metrics[train_metrics['label']==1].head()

,user_id,cate,shop_id,label,pred_prob
505,209632,34,3105,1.0,0.648109
718,34256,34,3939,1.0,0.368398
753,1386821,34,3939,1.0,0.820089
862,194940,34,3939,1.0,0.652782
972,1448839,34,3939,1.0,0.679566


In [93]:
help(StratifiedKFold)

Help on class StratifiedKFold in module sklearn.model_selection._split:

class StratifiedKFold(_BaseKFold)
 |  Stratified K-Folds cross-validator
 |  
 |  Provides train/test indices to split data in train/test sets.
 |  
 |  This cross-validation object is a variation of KFold that returns
 |  stratified folds. The folds are made by preserving the percentage of
 |  samples for each class.
 |  
 |  Read more in the :ref:`User Guide <cross_validation>`.
 |  
 |  Parameters
 |  ----------
 |  n_splits : int, default=3
 |      Number of folds. Must be at least 2.
 |  
 |  shuffle : boolean, optional
 |      Whether to shuffle each stratification of the data before splitting
 |      into batches.
 |  
 |  random_state : int, RandomState instance or None, optional, default=None
 |      If int, random_state is the seed used by the random number generator;
 |      If RandomState instance, random_state is the random number generator;
 |      If None, the random number generator is the RandomSt